In [1]:
%AddJar file:/data/home/vladimir.dyukarev/spark-cassandra-connector_2.11-2.4.0.jar
%AddJar file:/data/home/vladimir.dyukarev/postgresql-42.2.13.jar
%AddJar file:/data/home/vladimir.dyukarev/elasticsearch-spark-20_2.11-7.6.2.jar
%AddJar file:/data/home/vladimir.dyukarev/circe-core_2.11-0.11.2.jar

Using cached version of spark-cassandra-connector_2.11-2.4.0.jar
Using cached version of postgresql-42.2.13.jar
Using cached version of elasticsearch-spark-20_2.11-7.6.2.jar
Using cached version of circe-core_2.11-0.11.2.jar


#### Кассандра: клиенты

In [2]:
spark.conf.set("spark.cassandra.connection.host", "10.0.1.9")
spark.conf.set("spark.cassandra.connection.port", "9042")
spark.conf.set("spark.cassandra.output.consistency.level", "ANY")
spark.conf.set("spark.cassandra.input.consistency.level", "ONE")

In [3]:
val myTable = Map("table" -> "clients","keyspace" -> "labdata")
val dfClients = spark
    .read
    .format("org.apache.spark.sql.cassandra")
    .options(myTable)
    .load()

myTable = Map(table -> clients, keyspace -> labdata)
dfClients = [uid: string, age: int ... 1 more field]


[uid: string, age: int ... 1 more field]

In [4]:
dfClients .show(2, 200, true)

-RECORD 0--------------------------------------
 uid    | b9068896-2724-410b-b875-2e8c25dee8c0 
 age    | 41                                   
 gender | F                                    
-RECORD 1--------------------------------------
 uid    | 0526f8cb-be15-4fce-8947-03f42d81b0fd 
 age    | 29                                   
 gender | F                                    
only showing top 2 rows



#### Elastic: Логи посещения интернет-магазина

In [5]:
import org.apache.spark.sql.functions._

val esOptions = 
    Map(
        "es.nodes" -> "10.0.1.9:9200", 
        "es.batch.write.refresh" -> "false",
        "es.nodes.wan.only" -> "true"   
    )
val visits = spark.read.format("es").options(esOptions).load("visits")

esOptions = Map(es.nodes -> 10.0.1.9:9200, es.batch.write.refresh -> false, es.nodes.wan.only -> true)
visits = [category: string, event_type: string ... 4 more fields]


[category: string, event_type: string ... 4 more fields]

In [6]:
visits.show(5)

+--------------------+----------+--------------------+----------+-------------------+--------------------+
|            category|event_type|             item_id|item_price|          timestamp|                 uid|
+--------------------+----------+--------------------+----------+-------------------+--------------------+
|Entertainment-equ...|       buy|Entertainment-equ...|      2529|2020-01-01 11:00:00|40b29579-e845-45c...|
|    Everyday-jewelry|       buy|  Everyday-jewelry-0|      4320|2020-01-01 11:01:00|                null|
|             Cameras|      view|           Cameras-1|      1856|2020-01-01 11:02:00|ab0e7dd1-5899-488...|
|             Luggage|      view|           Luggage-7|      4975|2020-01-01 11:02:00|                null|
|       Mobile-phones|      view|    Mobile-phones-10|      3981|2020-01-01 11:03:00|d1206327-0e9f-410...|
+--------------------+----------+--------------------+----------+-------------------+--------------------+
only showing top 5 rows



#### Parquet на HDFS: Логи посещения веб-сайтов

In [7]:
import sys.process._
println("hdfs dfs -ls /labs/laba03/weblogs.json".!!)

-rw-r--r--   3 hdfs hdfs  563826904 2020-05-14 18:12 /labs/laba03/weblogs.json



In [8]:
val logs = spark.read.json("hdfs:///labs/laba03/weblogs.json")

logs = [uid: string, visits: array<struct<timestamp:bigint,url:string>>]


error: error while loading QualifiedTableName, class file '/usr/hdp/3.1.4.0-315/spark2/jars/spark-catalyst_2.11-2.4.5.jar(org/apache/spark/sql/catalyst/QualifiedTableName.class)' has location not matching its contents: contains class QualifiedTableName
error: error while loading JavaTypeInference, class file '/usr/hdp/3.1.4.0-315/spark2/jars/spark-catalyst_2.11-2.4.5.jar(org/apache/spark/sql/catalyst/JavaTypeInference.class)' has location not matching its contents: contains class JavaTypeInference
error: error while loading FunctionIdentifier, class file '/usr/hdp/3.1.4.0-315/spark2/jars/spark-catalyst_2.11-2.4.5.jar(org/apache/spark/sql/catalyst/FunctionIdentifier.class)' has location not matching its contents: contains class FunctionIdentifier
error: error while loading DefinedByConstructorParams, class file '/usr/hdp/3.1.4.0-315/spark2/jars/spark-catalyst_2.11-2.4.5.jar(org/apache/spark/sql/catalyst/DefinedByConstructorParams.class)' has location not matching its contents: contains 

[uid: string, visits: array<struct<timestamp:bigint,url:string>>]

In [11]:
logs.count

36138

#### PostgreSQL: категории сайтов

In [12]:
val jdbcUrl = "jdbc:postgresql://10.0.1.9:5432/labdata?user=vladimir_dyukarev&password=A3KQleIJ"

val catg = spark
    .read
    .format("jdbc")
    .option("url", jdbcUrl)
    .option("driver", "org.postgresql.Driver")
    .option("dbtable", "domain_cats")
    .load()

jdbcUrl = jdbc:postgresql://10.0.1.9:5432/labdata?user=vladimir_dyukarev&password=A3KQleIJ
catg = [domain: string, category: string]


[domain: string, category: string]

In [13]:
catg.show()

+--------------------+--------+
|              domain|category|
+--------------------+--------+
|national-lottery....|gambling|
|          rivalo.com|gambling|
|             xoso.me|gambling|
|         betin.co.ke|gambling|
|          skybet.com|gambling|
|              fdj.fr|gambling|
|     minhngoc.net.vn|gambling|
|   hollywoodbets.net|gambling|
|           tipico.de|gambling|
|       betpawa.co.ke|gambling|
|           bet365.it|gambling|
|          bet365.com|gambling|
|loteriasdominican...|gambling|
|            hkjc.com|gambling|
|        sunmaker.com|gambling|
|     williamhill.com|gambling|
|       parimatch.com|gambling|
|        freebitco.in|gambling|
|   mismarcadores.com|gambling|
|      flashscore.com|gambling|
+--------------------+--------+
only showing top 20 rows



Категории товаров из лога посещений интернет-магазина

In [14]:
//visits.show(2, 200 ,true)
visits.count

182540

In [39]:
visits.filter('uid === "40b29579-e845-45c0-a34d-03630d296a81").show(5, 200 ,true)

-RECORD 0------------------------------------------
 category   | Entertainment-equipment              
 event_type | buy                                  
 item_id    | Entertainment-equipment-2            
 item_price | 2529                                 
 timestamp  | 2020-01-01 11:00:00                  
 uid        | 40b29579-e845-45c0-a34d-03630d296a81 
-RECORD 1------------------------------------------
 category   | Computers                            
 event_type | buy                                  
 item_id    | Computers-6                          
 item_price | 4171                                 
 timestamp  | 2020-02-05 21:12:00                  
 uid        | 40b29579-e845-45c0-a34d-03630d296a81 
-RECORD 2------------------------------------------
 category   | Clothing                             
 event_type | view                                 
 item_id    | Clothing-19                          
 item_price | 2021                                 
 timestamp  

Фильтруем с помощью соединения с юзерами.
В колонках категорий товаров должно быть число посещений соответствующих страниц

In [15]:
val joinVisits = visits.join(dfClients, Seq("uid"), "inner")
        .filter('event_type === "view")
        .select('uid, 'category)

joinVisits = [uid: string, category: string]


[uid: string, category: string]

In [16]:
joinVisits.count

59996

In [17]:
// преобразуем категории
val udfvists = udf((x: String) => "shop_".concat(
    x.toLowerCase()
    .replace(' ','_')
    .replace('-','_')))

udfvists = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

In [18]:
val visits2 = joinVisits.withColumn("category2", udfvists(col("category"))).withColumn("1", lit(1))

visits2 = [uid: string, category: string ... 2 more fields]


[uid: string, category: string ... 2 more fields]

In [19]:
visits2.show(3,200, true)

-RECORD 0-----------------------------------------
 uid       | 0392f398-ea7e-4a1b-8c9d-d04df2a2a331 
 category  | Cameras                              
 category2 | shop_cameras                         
 1         | 1                                    
-RECORD 1-----------------------------------------
 uid       | 094b1e7e-97a6-4415-89f2-ca1eb72976b9 
 category  | Entertainment-equipment              
 category2 | shop_entertainment_equipment         
 1         | 1                                    
-RECORD 2-----------------------------------------
 uid       | 094b1e7e-97a6-4415-89f2-ca1eb72976b9 
 category  | Computers                            
 category2 | shop_computers                       
 1         | 1                                    
only showing top 3 rows



Сгруппировали и равернули, заменилл null на 0

In [20]:
val visits3 = visits2.groupBy("uid").pivot("category2").sum("1").na.fill(0)

visits3 = [uid: string, shop_cameras: bigint ... 15 more fields]


[uid: string, shop_cameras: bigint ... 15 more fields]

In [149]:
visits3.count

26453

In [22]:
visits3.filter('uid === "40b29579-e845-45c0-a34d-03630d296a81").show(3,300, true)

-RECORD 0--------------------------------------------------------------
 uid                            | 40b29579-e845-45c0-a34d-03630d296a81 
 shop_cameras                   | 0                                    
 shop_clothing                  | 1                                    
 shop_computers                 | 0                                    
 shop_cosmetics                 | 0                                    
 shop_entertainment_equipment   | 0                                    
 shop_everyday_jewelry          | 0                                    
 shop_house_repairs_paint_tools | 0                                    
 shop_household_appliances      | 1                                    
 shop_household_furniture       | 0                                    
 shop_kitchen_appliances        | 0                                    
 shop_kitchen_utensils          | 0                                    
 shop_luggage                   | 0                             

Категории веб-сайтов берутся из датасета категорий вебсайтов, и точно также из них создаются имена колонок. Например: web_arts_and_entertainment.

In [39]:
// логи веб-сайтов
logs. show(2, false)

+------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [137]:
//  джойн по клиентам, чтобы отсечь лишнее
val log_1 = logs.join(dfClients, Seq("uid"), "inner")
        .select('uid, 'visits)
val log_2  = log_1.select($"uid",explode($"visits")).select("uid", "col.url")


log_1 = [uid: string, visits: array<struct<timestamp:bigint,url:string>>]
log_2 = [uid: string, url: string]


[uid: string, url: string]

In [138]:
log_1.count

36138

In [91]:
log_1.show(false)

+------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|uid                                 |visits                                                                                                                                                                                                                                                                                                                                                

In [140]:
log_2.filter('uid === "d50192e5-c44e-4ae8-ae7a-7cfe67c8b777")show(3, 200, false)

+------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                 uid|                                                                                                                                                    url|
+------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+
|d50192e5-c44e-4ae8-ae7a-7cfe67c8b777|http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun|
|d50192e5-c44e-4ae8-ae7a-7cfe67c8b777|                                                 http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story|
|d50192e5-c44e-4ae8-ae7a-7cfe67c8b777|       

In [141]:
val log_3 = log_2.withColumn("host", callUDF("parse_url", $"url", lit("HOST")))
            .withColumn("host2", callUDF("replace",$"host" , lit("www."), lit("")))

log_3 = [uid: string, url: string ... 2 more fields]


[uid: string, url: string ... 2 more fields]

In [142]:
log_3.show(3)

+--------------------+--------------------+-------------------+---------------+
|                 uid|                 url|               host|          host2|
+--------------------+--------------------+-------------------+---------------+
|0392f398-ea7e-4a1...|http://www.adme.r...|        www.adme.ru|        adme.ru|
|0392f398-ea7e-4a1...|http://www.adme.r...|        www.adme.ru|        adme.ru|
|0392f398-ea7e-4a1...|http://www.online...|www.onlinesudoku.ru|onlinesudoku.ru|
+--------------------+--------------------+-------------------+---------------+
only showing top 3 rows



In [111]:
// категории
catg.groupBy("category").count.show(false)

+------------------------+-----+
|category                |count|
+------------------------+-----+
|travel                  |318  |
|business_and_industry   |9610 |
|finance                 |40179|
|computer_and_electronics|506  |
|internet_and_telecom    |1015 |
|arts_and_entertainment  |48522|
|shopping                |44428|
|law_and_government      |185  |
|beauty_and_fitness      |21026|
|home_and_garden         |79   |
|recreation_and_hobbies  |53   |
|autos_and_vehicles      |183  |
|games                   |393  |
|science                 |53   |
|books_and_literature    |115  |
|gambling                |9829 |
|reference               |125  |
|health                  |34950|
|food_and_drink          |96   |
|sports                  |185  |
+------------------------+-----+
only showing top 20 rows



In [143]:
// джойним категории

import org.apache.spark.sql.Column
val joinCondition: Column = col("host") === col("domain")

val joinSite = log_3.join(catg, joinCondition)

joinCondition = (host = domain)
joinSite = [uid: string, url: string ... 4 more fields]


[uid: string, url: string ... 4 more fields]

In [144]:
// преобразуем категории
val udflogs = udf((x: String) => "web_".concat(
    x.toLowerCase()
    .replace(' ','_')
    .replace('-','_')))

udflogs = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

In [145]:
val joinSite2 = joinSite
            .withColumn("category2", udflogs(col("category")))
            .withColumn("1", lit(1))

joinSite2 = [uid: string, url: string ... 6 more fields]


[uid: string, url: string ... 6 more fields]

In [147]:
val joinSite3 = joinSite2.groupBy("uid").pivot("category2").sum("1").na.fill(0)

joinSite3 = [uid: string, web_arts_and_entertainment: bigint ... 21 more fields]


[uid: string, web_arts_and_entertainment: bigint ... 21 more fields]

In [148]:
joinSite3.show(5)

+--------------------+--------------------------+----------------------+----------------------+------------------------+-------------------------+------------------------+----------------------------+-----------+------------------+------------+---------+----------+-------------------+------------------------+----------------------+------------------+--------------------------+-------------+-----------+------------+----------+----------+
|                 uid|web_arts_and_entertainment|web_autos_and_vehicles|web_beauty_and_fitness|web_books_and_literature|web_business_and_industry|web_career_and_education|web_computer_and_electronics|web_finance|web_food_and_drink|web_gambling|web_games|web_health|web_home_and_garden|web_internet_and_telecom|web_law_and_government|web_news_and_media|web_recreation_and_hobbies|web_reference|web_science|web_shopping|web_sports|web_travel|
+--------------------+--------------------------+----------------------+----------------------+-----------------------

### Соединяем все в одну витрину

In [151]:
val all_final = dfClients.join(visits3, Seq("uid"), "left")
        .join(joinSite3, Seq("uid"), "left")

all_final = [uid: string, age: int ... 39 more fields]


[uid: string, age: int ... 39 more fields]

In [154]:
all_final.show(1, 300, false)

+------------------------------------+---+------+------------+-------------+--------------+--------------+----------------------------+---------------------+------------------------------+-------------------------+------------------------+-----------------------+---------------------+------------+------------------+----------+---------------------+---------+--------------------------+----------------------+----------------------+------------------------+-------------------------+------------------------+----------------------------+-----------+------------------+------------+---------+----------+-------------------+------------------------+----------------------+------------------+--------------------------+-------------+-----------+------------+----------+----------+
|                                 uid|age|gender|shop_cameras|shop_clothing|shop_computers|shop_cosmetics|shop_entertainment_equipment|shop_everyday_jewelry|shop_house_repairs_paint_tools|shop_household_appliances|shop_ho

In [156]:
all_final.count

36138

In [155]:
dfClients.count

36138